# Inception V4

In [ ]:
name = 'inceptionv4'

In [ ]:
# Import needed libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import utilities
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization, Activation, AveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
# Check Tensorflow version
print('Tensorflow version: ',tf.__version__)
# Check if GPU is being used
print('GPU available: ', tf.config.list_physical_devices('GPU'))
# If a GPU is being used, check that it's not being used by another process as well
if tf.config.list_physical_devices('GPU'):
  print("GPU usage: ", tf.config.experimental.get_memory_info('GPU:0'))

## Load the data

In [ ]:
def process_image(path, label):
    # Desired size
    size = 250
    # Get the image
    img = tf.io.read_file(path)
    # Decode the PNG
    img = tf.image.decode_png(img)
    # Resize image
    img = tf.image.resize(img, (size, size))
    # Reshape image (this is not necessary but I do it so that I don't need to be modifying the shape in the input layer)
    img = tf.reshape(img, [size, size, 1])
    # Cast image to float32
    img = tf.cast(img, tf.float32)
    # Normalize image
    img = img/255.0

    return img, label

In [ ]:
def get_paths_n_labels(csv_path, images_folder_path, id_col, label_col):
    df = pd.read_csv(csv_path)
    paths = [images_folder_path + '/' + str(id) + '.png' for id in df[id_col].tolist()]
    labels = df[label_col].tolist()
    return paths, labels

In [ ]:
# Dataset used
ds = 'clahecropfill2_r250'

# Input pipeline
# Training set
train_paths, train_labels = get_paths_n_labels( csv_path='./data/pre_processed/training/train.csv',
                                                images_folder_path='./data/pre_processed/training/{}'.format(ds),
                                                id_col='id',
                                                label_col='boneage')
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_dataset = train_dataset.map(process_image)
train_dataset = train_dataset.shuffle(len(train_dataset)).batch(8)

#Validation set
validation_paths, validation_labels = get_paths_n_labels(
                                                csv_path='./data/pre_processed/validation/validation/validation.csv',
                                                images_folder_path='./data/pre_processed/validation/validation/{}'.format(ds),
                                                id_col='Image ID',
                                                label_col='Bone Age (months)')
validation_dataset = tf.data.Dataset.from_tensor_slices((validation_paths, validation_labels))
validation_dataset = validation_dataset.map(process_image).batch(8)

In [ ]:
# for img, label in train_dataset:
#     print(type(img))
#     break

## Instantiate the model

Useful functions

In [ ]:
#-----------------------------------------------
# Convolution + Batch Normalization + Activation
#-----------------------------------------------
# conv_bn_act block
#-----------------------------------------------
def conv_bn_act(X_input, filters, kernel_size, strides, padding='same', activation=None, name=None):

    # Add the layers
    x = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = padding, name='conv_'+name)(X_input)
    x  = BatchNormalization(name='bn'+name)(x)
    if activation is not None:
        x = Activation(activation)(x)
    return x

Inception blocks

In [ ]:
#-----------------------------------------------
# STEM BLOCK
#-----------------------------------------------
# stem_block
#-----------------------------------------------
def stem_block(X_input):

    # First conv
    X = conv_bn_act(X_input, filters = 32, kernel_size = (3, 3), strides = (2, 2),
                    padding = 'valid', activation='relu', name = 'stem_1th')
    # Second conv
    X = conv_bn_act(X, filters = 32, kernel_size = (3, 3), strides = (1, 1),
                    padding = 'valid', activation='relu', name = 'stem_2nd')
    # Third conv
    X = conv_bn_act(X, filters = 64, kernel_size = (3, 3), strides = (1, 1),
                  padding = 'same', activation='relu', name =  'stem_3rd')
    #-----------------------------------------------
    # BRANCH SPLIT
    #-----------------------------------------------
    # First branch: max pooling
    branch1 = MaxPooling2D(pool_size = (3, 3), strides = (2, 2),
                           padding = 'valid', name = 'stem_1stbranch_1')(X)
    # Second branch: conv
    branch2 = conv_bn_act(X, filters = 96, kernel_size = (3, 3), strides = (2, 2),
                          padding = 'valid', activation='relu', name = 'stem_1stbranch_2')

    #-----------------------------------------------
    # MERGE BRANCHES
    #-----------------------------------------------
    # Concatenate (1) branch1 and branch2 along the channel axis
    X = tf.concat(values=[branch1, branch2], axis=3)

    #-----------------------------------------------
    # BRANCH SPLIT
    #-----------------------------------------------
    # First branch: 2 convs
    branch1 = conv_bn_act(X, filters = 64, kernel_size = (1, 1),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = 'stem_2ndbranch_1_1')
    branch1 = conv_bn_act(branch1, filters = 96, kernel_size = (3, 3),
                        strides = (1, 1), padding = 'valid', activation='relu',
                        name = 'stem_2ndbranch_1_2')

    # Second branch: 4 convs
    branch2 = conv_bn_act(X, filters = 64, kernel_size = (1, 1),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = 'stem_2ndbranch_2_1')
    branch2 = conv_bn_act(branch2, filters = 64, kernel_size = (7, 1),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = 'stem_2ndbranch_2_2')
    branch2 = conv_bn_act(branch2, filters = 64, kernel_size = (1, 7),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = 'stem_2ndbranch_2_3')
    branch2 = conv_bn_act(branch2, filters = 96, kernel_size = (3, 3),
                        strides = (1, 1), padding = 'valid', activation='relu',
                        name = 'stem_2ndbranch_2_4')

    #-----------------------------------------------
    # MERGE BRANCHES
    #-----------------------------------------------
    # Concatenate (2) branch1 and branch2 along the channel axis
    X = tf.concat(values=[branch1, branch2], axis=3)

    #-----------------------------------------------
    # BRANCH SPLIT
    #-----------------------------------------------
    # First branch: conv
    branch1 = conv_bn_act(X, filters = 192, kernel_size = (3, 3), strides = (2, 2),
                          padding = 'valid', activation='relu', name = 'stem_3rdbranch_1')
    # Second branch: max pooling
    branch2 = MaxPooling2D(pool_size = (3, 3), strides = (2, 2),
                           padding = 'valid', name = 'stem_3rdbranch_2')(X)

    #-----------------------------------------------
    # MERGE BRANCHES
    #-----------------------------------------------
    # Concatenate (3) branch1 and branch2 along the channel axis
    X = tf.concat(values=[branch1, branch2], axis=3)

    return X

In [ ]:
#-----------------------------------------------
# INCEPTION A BLOCK
#-----------------------------------------------
# inception_a_block
#-----------------------------------------------
def inception_a_block(X_input, base_name):

    #-----------------------------------------------
    # BRANCH SPLIT
    #-----------------------------------------------
    # Branch 1
    branch1 = AveragePooling2D(pool_size = (3, 3), strides = (1, 1),
                               padding = 'same', name = base_name + 'ia_branch_1_1')(X_input)
    branch1 = conv_bn_act(branch1, filters = 96, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ia_branch_1_2')
    # Branch 2
    branch2 = conv_bn_act(X_input, filters = 96, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ia_branch_2_1')
    # Branch 3
    branch3 = conv_bn_act(X_input, filters = 64, kernel_size = (1, 1),strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ia_branch_3_1')
    branch3 = conv_bn_act(branch3, filters = 96, kernel_size = (3, 3), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ia_branch_3_2')
    # Branch 4
    branch4 = conv_bn_act(X_input, filters = 64, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ia_branch_4_1')
    branch4 = conv_bn_act(branch4, filters = 96, kernel_size = (3, 3), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ia_branch_4_2')
    branch4 = conv_bn_act(branch4, filters = 96, kernel_size = (3, 3), strides = (1, 1),
                          padding = 'same', activation='relu',  name = base_name + 'ia_branch_4_3')

    #-----------------------------------------------
    # MERGE BRANCHES
    #-----------------------------------------------
    # Concatenate branch1, branch2, branch3 and branch4 along the channel axis
    X = tf.concat(values=[branch1, branch2, branch3, branch4], axis=3)

    return X

In [ ]:
#-----------------------------------------------
# INCEPTION B BLOCK
#-----------------------------------------------
# inception_b_block
#-----------------------------------------------
def inception_b_block(X_input, base_name):

    #-----------------------------------------------
    # BRANCH SPLIT
    #-----------------------------------------------
    # Branch 1
    branch1 = AveragePooling2D(pool_size = (3, 3), strides = (1, 1),
                           padding = 'same', name = base_name + 'ib_branch_1_1')(X_input)
    branch1 = conv_bn_act(branch1, filters = 128, kernel_size = (1, 1),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = base_name + 'ib_branch_1_2')
    # Branch 2
    branch2 = conv_bn_act(X_input, filters = 384, kernel_size = (1, 1),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = base_name + 'ib_branch_2_1')
    # Branch 3
    branch3 = conv_bn_act(X_input, filters = 192, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ib_branch_3_1')
    branch3 = conv_bn_act(branch3, filters = 224, kernel_size = (1, 7), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ib_branch_3_2')
    branch3 = conv_bn_act(branch3, filters = 256, kernel_size = (7, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ib_branch_3_3')

    # Branch 4
    branch4 = conv_bn_act(X_input, filters = 192, kernel_size = (1, 1),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = base_name + 'ib_branch_4_1')
    branch4 = conv_bn_act(branch4, filters = 192, kernel_size = (1, 7),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = base_name + 'ib_branch_4_2')
    branch4 = conv_bn_act(branch4, filters = 224, kernel_size = (7, 1),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = base_name + 'ib_branch_4_3')
    branch4 = conv_bn_act(branch4, filters = 224, kernel_size = (1, 7),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = base_name + 'ib_branch_4_4')
    branch4 = conv_bn_act(branch4, filters = 256, kernel_size = (7, 1),
                        strides = (1, 1), padding = 'same', activation='relu',
                        name = base_name + 'ib_branch_4_5')

    #-----------------------------------------------
    # MERGE BRANCHES
    #-----------------------------------------------
    # Concatenate branch1, branch2, branch3 and branch4 along the channel axis
    X = tf.concat(values=[branch1, branch2, branch3, branch4], axis=3)

    return X

In [ ]:
#-----------------------------------------------
# INCEPTION C BLOCK
#-----------------------------------------------
# inception_c_block
#-----------------------------------------------
def inception_c_block(X_input, base_name):

    #-----------------------------------------------
    # BRANCH SPLIT
    #-----------------------------------------------

    # Branch 1
    branch1 = AveragePooling2D(pool_size = (3, 3), strides = (1, 1),
                           padding = 'same', name = base_name + 'ic_branch_1_1')(X_input)
    branch1 = conv_bn_act(branch1, filters = 256, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ic_branch_1_2')

    # Branch 2
    branch2 = conv_bn_act(X_input, filters = 256, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ic_branch_2_1')

    # Branch 3
    branch3 = conv_bn_act(X_input, filters = 384, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ic_branch_3_1')
    branch3_1 = conv_bn_act(branch3, filters = 256, kernel_size = (1, 3), strides = (1, 1),
                            padding = 'same', activation='relu', name = base_name + 'ic_branch_3_2')
    branch3_2 = conv_bn_act(branch3, filters = 256, kernel_size = (3, 1), strides = (1, 1),
                            padding = 'same', activation='relu', name = base_name + 'ic_branch_3_3')

    # Branch 4
    branch4 = conv_bn_act(X_input, filters = 384, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ic_branch_4_1')
    branch4 = conv_bn_act(branch4, filters = 448, kernel_size = (1, 3), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ic_branch_4_2')
    branch4 = conv_bn_act(branch4, filters = 512, kernel_size = (3, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ic_branch_4_3')
    branch4_1 = conv_bn_act(branch4, filters = 256, kernel_size = (3, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = base_name + 'ic_branch_4_4')
    branch4_2 = conv_bn_act(branch4, filters = 256, kernel_size = (1, 3), strides = (1, 1),
                            padding = 'same', activation='relu', name = base_name + 'ic_branch_4_5')

    #-----------------------------------------------
    # MERGE BRANCHES
    #-----------------------------------------------
    # Concatenate branch1, branch2, branch3_1, branch3_2, branch4_1 and branch4_2 along the channel axis
    X = tf.concat(values=[branch1, branch2, branch3_1, branch3_2, branch4_1, branch4_2], axis=3)

    return X

In [ ]:
#-----------------------------------------------
# REDUCTION A BLOCK
#-----------------------------------------------
# reduction_a_block
#-----------------------------------------------
def reduction_a_block(X_input):
    #-----------------------------------------------
    # BRANCH SPLIT
    #-----------------------------------------------
    # Branch 1
    branch1 = MaxPooling2D(pool_size = (3, 3), strides = (2, 2),
                           padding = 'valid', name = 'ra_branch_1_1')(X_input)
    # Branch 2
    branch2 = conv_bn_act(X_input, filters = 384, kernel_size = (3, 3), strides = (2, 2),
                          padding = 'valid', activation='relu', name = 'ra_branch_2_1')
    # Branch 3
    branch3 = conv_bn_act(X_input, filters = 192, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = 'ra_branch_3_1')
    branch3 = conv_bn_act(branch3, filters = 224, kernel_size = (3, 3), strides = (1, 1),
                          padding = 'same', activation='relu', name = 'ra_branch_3_2')
    branch3 = conv_bn_act(branch3, filters = 256, kernel_size = (3, 3), strides = (2, 2),
                          padding = 'valid', activation='relu',name = 'ra_branch_3_3')

    #-----------------------------------------------
    # MERGE BRANCHES
    #-----------------------------------------------
    # Concatenate branch1, branch2 and branch3 along the channel axis
    X = tf.concat(values=[branch1, branch2, branch3], axis=3)

    return X

In [ ]:
#-----------------------------------------------
# REDUCTION B BLOCK
#-----------------------------------------------
# reduction_b_block
#-----------------------------------------------
def reduction_b_block(X_input):
    #-----------------------------------------------
    # BRANCH SPLIT
    #-----------------------------------------------
    # Branch 1
    branch1 = MaxPooling2D(pool_size = (3, 3), strides = (2, 2),
                           padding = 'valid', name = 'rb_branch_1_1')(X_input)

    # Branch 2
    branch2 = conv_bn_act(X_input, filters = 192, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = 'rb_branch_2_1')
    branch2 = conv_bn_act(branch2, filters = 192, kernel_size = (3, 3), strides = (2, 2),
                          padding = 'valid', activation='relu', name = 'rb_branch_2_2')

    # Branch 3
    branch3 = conv_bn_act(X_input, filters = 256, kernel_size = (1, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = 'rb_branch_3_1')
    branch3 = conv_bn_act(branch3, filters = 256, kernel_size = (1, 7), strides = (1, 1),
                          padding = 'same', activation='relu', name = 'rb_branch_3_2')
    branch3 = conv_bn_act(branch3, filters = 320, kernel_size = (7, 1), strides = (1, 1),
                          padding = 'same', activation='relu', name = 'rb_branch_3_3')
    branch3 = conv_bn_act(branch3, filters = 320, kernel_size = (3, 3), strides = (2, 2),
                          padding = 'valid', activation='relu', name = 'rb_branch_3_4')

    #-----------------------------------------------
    # MERGE BRANCHES
    #-----------------------------------------------
    # Concatenate branch1, branch2 and branch3 along the channel axis
    X = tf.concat(values=[branch1, branch2, branch3], axis=3)

    return X

Network construction

In [ ]:
# FUNCTION: Inception-v4

def Inceptionv4(input_shape):

    # Define the input as a tensor with shape input_shape (1 line)
    X_input = Input(shape=input_shape)

    # Call the above functions for the stem, inception-a, reduction-a, inception-b, reduction-b and inception-c blocks
    X = stem_block(X_input)

    # Four Inception A blocks
    X = inception_a_block(X, 'a1')
    X = inception_a_block(X, 'a2')
    X = inception_a_block(X, 'a3')
    X = inception_a_block(X, 'a4')

    # Reduction A block
    X = reduction_a_block(X)

    # Seven Inception B blocks
    X = inception_b_block(X, 'b1')
    X = inception_b_block(X, 'b2')
    X = inception_b_block(X, 'b3')
    X = inception_b_block(X, 'b4')
    X = inception_b_block(X, 'b5')
    X = inception_b_block(X, 'b6')
    X = inception_b_block(X, 'b7')

    # Reduction B block
    X = reduction_b_block(X)

    # Three Inception C blocks
    X = inception_c_block(X, 'c1')
    X = inception_c_block(X, 'c2')
    X = inception_c_block(X, 'c3')

    # AVGPOOL (1 line). Use "X = AveragePooling2D(...)(X)"
    kernel_pooling = X.get_shape()[1:3]
    X = AveragePooling2D(kernel_pooling, name='avg_pool')(X)
    X = Flatten()(X)

    # Dropout
    X = Dropout(rate = 0.2)(X)

    # Output layer
    X = Dense(1, name='linear')(X)

    # Create model
    model = Model(inputs = X_input, outputs = X, name='Inceptionv4')

    return model

In [ ]:
model = Inceptionv4(input_shape = (250, 250, 1))
optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)

# Compile the model
model.compile(optimizer=optimizer,
              loss='mse',
              metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
model.summary()

In [ ]:
# Experiment tracking with tensorboard
import time
experiment_name = "{}_{}_{}".format(name,ds,int(time.time()))
tensorboard = tf.keras.callbacks.TensorBoard(log_dir='./logs/{}'.format(experiment_name))

## Training ("fit") the model

In [ ]:
# Fit the model
r = model.fit(train_dataset, validation_data=validation_dataset, epochs=100, callbacks=[tensorboard])

## Save the model

In [ ]:
# Let's now save our model to a file
model.save('./models/{}.h5'.format(experiment_name))

## Make predictions

In [ ]:
# Load the model
model = tf.keras.models.load_model('./models/{}.h5'.format(experiment_name))
model.evaluate(validation_dataset)

In [ ]:
p_test = model.predict(validation_dataset).reshape(-1)

In [ ]:
# Show some misclassified examples
y_test = np.array([y.numpy() for x, y in validation_dataset.unbatch()])
x_test = np.array([x.numpy() for x, y in validation_dataset.unbatch()])
difference = np.abs(np.subtract(y_test, p_test))
misclassified_idx = np.where(difference<4)
#print(misclassified_idx)
i = np.random.choice(misclassified_idx[0])
plt.imshow(x_test[i], cmap='gray')
plt.title("True label: %s Predicted: %s" % (y_test[i], p_test[i]));